In [16]:
import os
import pandas as pd
import json
import traceback
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

In [ ]:
# from langchain.chat_models import ChatOpenAI

In [13]:
load_dotenv()
KEY = os.getenv("OPENAI_API_KEY")

In [18]:
# llm = ChatOpenAI(openai_api_key = KEY, model_name="gpt-3.5-turbo", temperature=0.5)
llm = init_chat_model(model="gpt-3.5-turbo", model_provider="openai")


In [19]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001E867C95CA0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001E867C95DF0>, root_client=<openai.OpenAI object at 0x000001E866A56280>, root_async_client=<openai.AsyncOpenAI object at 0x000001E867C95970>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [20]:
from langchain_openai import OpenAI
from langchain.callbacks import get_openai_callback
from langchain_core.prompts import PromptTemplate

In [ ]:
#quiz prompt template
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

TEMPLATE = """
Text: {text}

You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. \
Make sure the questions are not repeated and check all the questions to be conforming to the text as well. \
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs

### RESPONSE_JSON
{response_json}
"""


quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template= TEMPLATE
)

In [ ]:
# first llm chain
from langchain.chains import LLMChain
quiz_chain = LLMChain(llm= llm, prompt = quiz_generation_prompt, output_key="quiz", verbose=True)

In [25]:
# quiz evalutation prompt

TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

# second chain, review chain
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [26]:
# Sequential chain to join both chains
from langchain.chains import SequentialChain
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [ ]:
# reading the data.txt
file_path = r"C:\Users\adars\Documents\MyDocuments\FAU\ML_Profile\LLM_30_hrs\MCQ_Generator_Project\data.txt"

with open(file_path, "r") as file:
    TEXT = file.read()

In [28]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [30]:
# json serializer to serialize the dictionary into json-format-string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [32]:
# to do the token usage tracking in Langchain (nowadays langsmith is used)
NUMBER = 5
SUBJECT = "machine learning"
TONE= "simple" # diffculty of mcq

with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\adars\AppData\Local\Temp\ipykernel_93908\1527886663.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text: Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text: Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools

In [33]:
# token tracking using get_openai_callback()
print(f"Total Tokens: {cb.total_tokens}")
print(f"Input Tokens: {cb.prompt_tokens}")
print(f"Output Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 1845
Input Tokens: 1186
Output Tokens: 659
Total Cost: 0.0015815


In [36]:
quiz = response.get("quiz")

In [41]:
quiz = json.loads(quiz) # the output is formatted in the json format, because json-serialized-string is the output type
quiz

{'1': {'mcq': 'What is biology?',
  'options': {'a': 'Study of rocks',
   'b': 'Study of life',
   'c': 'Study of planets',
   'd': 'Study of computers'},
  'correct': 'b'},
 '2': {'mcq': 'What is a major theme in biology that explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Chemistry',
   'c': 'Physics',
   'd': 'Mathematics'},
  'correct': 'a'},
 '3': {'mcq': 'What allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Energy processing',
   'b': 'Sleeping',
   'c': 'Studying',
   'd': 'Eating'},
  'correct': 'a'},
 '4': {'mcq': 'What is the scientific method used for in biology?',
  'options': {'a': 'Playing games',
   'b': 'Making observations',
   'c': 'Watching movies',
   'd': 'Listening to music'},
  'correct': 'b'},
 '5': {'mcq': 'When did life on Earth emerge?',
  'options': {'a': '1 million years ago',
   'b': '100 million years ago',
   'c': '3.7 billion years ago',
   'd': '10 billion years ago'},
  'correct': 'c'}}

In [39]:
json.loads(response.get("review"))

{'1': {'mcq': 'What is the scientific study of life called?',
  'options': {'a': 'Chemistry',
   'b': 'Biology',
   'c': 'Physics',
   'd': 'Geology'},
  'correct': 'b'},
 '2': {'mcq': 'Which theme explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Energy processing',
   'c': 'Cell structure',
   'd': 'Genetic information'},
  'correct': 'a'},
 '3': {'mcq': 'What allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Evolution',
   'b': 'Energy processing',
   'c': 'Cell structure',
   'd': 'Genetic information'},
  'correct': 'b'},
 '4': {'mcq': 'What is the process that biologists use to make observations and form conclusions?',
  'options': {'a': 'Guessing',
   'b': 'Intuition',
   'c': 'Scientific method',
   'd': 'Magic'},
  'correct': 'c'},
 '5': {'mcq': 'How long ago did life on Earth emerge?',
  'options': {'a': '1 billion years ago',
   'b': '3.7 billion years ago',
   'c': '5 billion years ago',
   'd': '10 billion years ago'},


In [42]:
## storing the quiz data to csv file
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})
    
# dataframe
quiz=pd.DataFrame(quiz_table_data)

quiz.to_csv("machinelearning.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S') #string formatting the datetime

'05_18_2025_13_16_20'